In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
## Vector Embedding And Vector Store
from langchain.vectorstores import FAISS, Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import os
load_dotenv()

## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

llm = ChatOllama(model='llama3.1:8b', temperature=0)

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

ollama_emb = OllamaEmbeddings(model="llama3.1:8b")

faiss_db = FAISS.from_documents(documents, ollama_emb)
chroma_db = Chroma.from_documents(documents, ollama_emb)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
retriever=faiss_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12c3dc7d0>)

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(
	retriever,
	"langsmith_search",
	"Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [7]:
retriever_tool.name

'langsmith_search'

In [8]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools=[wiki,arxiv,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/sanketughadmathe/.pyenv/versions/3.12.2/envs/GenAI/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x12a2d85e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorsto

In [11]:
from langchain import hub
# Get the prompt to use
prompt = hub.pull("hwchase17/openai-functions-agent")

/Users/sanketughadmathe/.pyenv/versions/3.12.2/envs/GenAI/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [12]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [13]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [14]:
agent_executor.invoke({"input":"Tell me about Langchain"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langchain'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. L

{'input': 'Tell me about Langchain',
 'output': 'Based on the tool call response, I can format an answer to the original question "Tell me about Langchain".\n\nLangchain is not explicitly mentioned in the provided output. However, based on the context and information provided, it appears that Langchain is a platform for building production-grade LLM (Large Language Model) applications. It allows users to closely monitor and evaluate their application, enabling them to ship quickly and with confidence.\n\nThe output suggests that Langchain can be used in conjunction with LangSmith, which is a separate platform for building and evaluating LLM applications. However, it\'s also mentioned that using LangChain is not necessary, as LangSmith works on its own.\n\nTherefore, the answer to the original question would be:\n\nLangchain appears to be a platform for building production-grade LLM applications, closely related to or possibly a part of the LangSmith ecosystem.'}

In [15]:
agent_executor.invoke({"input":"Tell me about ML"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Machine Learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton UniverBased on the tool call response, here's an answer to your original question:

Machine Learning (ML) is a field of study that focuses on developing algorithms and statistical models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. The lecture notes mentioned in the tool call response provide a comprehensive overview of optimization techniques for machine learning, which are essential for training ML models.

Some key concepts in ML include:

1. Supervised Learning: This involves training a model on labeled data to predict outcomes.
2. Unsupervised Learning: This involves training a model on unlabeled data to identify patterns or rel

{'input': 'Tell me about ML',
 'output': "Based on the tool call response, here's an answer to your original question:\n\nMachine Learning (ML) is a field of study that focuses on developing algorithms and statistical models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. The lecture notes mentioned in the tool call response provide a comprehensive overview of optimization techniques for machine learning, which are essential for training ML models.\n\nSome key concepts in ML include:\n\n1. Supervised Learning: This involves training a model on labeled data to predict outcomes.\n2. Unsupervised Learning: This involves training a model on unlabeled data to identify patterns or relationships.\n3. Deep Learning: This is a type of ML that uses neural networks with multiple layers to learn complex representations of data.\n\nThese concepts, and others like them, are crucial for understanding the field of Machine Learning."}

In [16]:
agent_executor.invoke({"input":"Tell me about Titanic"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Titanic'}`


Page: Titanic
Summary: RMS Titanic was a British ocean liner that sank on 15 April 1912 as a result of striking an iceberg on her maiden voyage from Southampton, England to New York City, United StateThe RMS Titanic was a British ocean liner that sank on April 15, 1912, after colliding with an iceberg during its maiden voyage. This tragic event resulted in the loss of over 1,500 lives and became one of the most infamous maritime disasters in history.

> Finished chain.


{'input': 'Tell me about Titanic',
 'output': 'The RMS Titanic was a British ocean liner that sank on April 15, 1912, after colliding with an iceberg during its maiden voyage. This tragic event resulted in the loss of over 1,500 lives and became one of the most infamous maritime disasters in history.'}